In [1]:
pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pandas_datareader import data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
# 取得する期間
start = '2019-06-01'
end = '2020-06-01'

df = data.DataReader('^N225','yahoo',start,end)

In [4]:
# 10個だけ表示
df.head(10)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-06-03,20438.029297,20305.740234,20327.869141,20410.880859,67400000,20410.880859
2019-06-04,20464.570312,20289.640625,20435.859375,20408.539062,69200000,20408.539062
2019-06-05,20800.640625,20646.150391,20667.890625,20776.099609,72300000,20776.099609
2019-06-06,20842.279297,20745.839844,20745.839844,20774.039062,59600000,20774.039062
2019-06-07,20907.769531,20816.580078,20859.779297,20884.710938,50700000,20884.710938
2019-06-10,21166.119141,21077.949219,21095.400391,21134.419922,58200000,21134.419922
2019-06-11,21227.179688,21066.619141,21099.539062,21204.279297,52100000,21204.279297
2019-06-12,21259.699219,21118.750000,21130.390625,21129.720703,56500000,21129.720703
2019-06-13,21111.230469,20932.130859,21040.910156,21032.000000,65700000,21032.000000


In [ ]:
列のcolum
行のindex
データ部分の三つから成り立っている

High→高値：その日の株価の動きの中で最も高い株価での取引が成立した価格
Low→安値：その日の株価の動きの中で最も安い株価での取引が成立した価格
Open→始値：その日の市場が開いてから最初に売買が成立して取引された時の株価
Close→終値：その日最後となった売買が成立した際に取引された株価
Volume→出来高：株取引された量※株価に先行する
Adj Close→調整後の終値：

In [ ]:
df.index

In [ ]:
data = df.index
price= df['Adj Close']

In [ ]:
plt.plot(data,price)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(data,price)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(data,price,label='Nikkei225')
plt.legend()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(data,price,label='Nikkei225')
plt.title('N225',color='blue',backgroundcolor='white',size=40,loc='center')
plt.legend()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(data,price,label='Nikkei225')
plt.title('N225',color='blue',backgroundcolor='white',size=40,loc='center')
plt.xlabel('data',color='black',size=30)
plt.ylabel('price',color='black',size=30)
plt.legend()

In [ ]:
#  ここから移動平均
span01=5
span02=25
span03=50

df['sma01']=price.rolling(window=span01).mean()
df['sma02']=price.rolling(window=span02).mean()
df['sma03']=price.rolling(window=span03).mean()

In [ ]:
pd.set_option('display.max_rows',None)
df.head(100)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(data,price,label='Nikkei225')
plt.plot(data,df['sma01'],label='sma01')
plt.plot(data,df['sma02'],label='sma02')
plt.plot(data,df['sma03'],label='sma03')

plt.title('N225',color='blue',backgroundcolor='white',size=40,loc='center')
plt.xlabel('data',color='black',size=30)
plt.ylabel('price',color='black',size=30)

plt.legend()

In [ ]:
# ゴールデンクロス：短期の移動平均線が中期以上の移動平均線を下から上に抜けること→価格が上昇しているサイン
# デットクロス：短期の移動平均線が中期以上の移動平均線を上から下に抜けること→価格が下降しているサイン

In [ ]:
plt.figure(figsize=(30,15))
plt.bar(data,df['Volume'],label='Volume',color='grey')

plt.legend()

In [ ]:
plt.figure(figsize=(30,10))
plt.subplot(2,1,1)
plt.plot(data,price,label='Close',color='#99b898')
plt.plot(data,df['sma01'],label='sma01',color='#e84a5f')
plt.plot(data,df['sma02'],label='sma02',color='#ff847c')
plt.plot(data,df['sma03'],label='sma03',color='#feceab')
plt.legend()

plt.subplot(2,1,2)
plt.bar(data,df['Volume'],label='Volume',color='grey')

plt.legend()

In [ ]:
# リクルートホールディングス
df = data.DataReader('6098.JP' , 'stooq')

In [ ]:
df.head(10)

In [ ]:
df.index.min()

In [ ]:
df.index.max()

In [ ]:
df= df.sort_index()

In [ ]:
df.head(10)

In [ ]:
df.index>='2019-06-01 00:00:00'

In [ ]:
df[df.index>='2019-06-01 00:00:00']

In [ ]:
df.index<='2020-06-01 00:00:00'

In [ ]:
df[df.index<='2020-06-01 00:00:00']

In [ ]:
pd.set_option('display.max_rows',None)
df[(df.index>='2019-06-01 00:00:00') & (df.index<='2020-05-01 00:00:00')]

In [ ]:
# リクルートホールディングス
df = data.DataReader('6098.JP' , 'stooq')

data=df.index
price=df['Close']

span01=5
span02=25
span03=50

df['sma01']=price.rolling(window=span01).mean()
df['sma02']=price.rolling(window=span02).mean()
df['sma03']=price.rolling(window=span03).mean()

plt.figure(figsize=(30,10))
plt.subplot(2,1,1)
plt.plot(data,price,label='Close',color='#99b898')
plt.plot(data,df['sma01'],label='sma01',color='#e84a5f')
plt.plot(data,df['sma02'],label='sma02',color='#ff847c')
plt.plot(data,df['sma03'],label='sma03',color='#feceab')
plt.legend()

plt.subplot(2,1,2)
plt.bar(data,df['Volume'],label='Volume',color='grey')
plt.legend()

In [ ]:
start = '2019-06-01'
end = '2020-05-01'
company_code = '7203.JP'

In [ ]:
df = data.DataReader(company_code,'stooq')
df = df[(df.index>=start) & (df.index<=end)]



data=df.index
price=df['Close']

span01=5
span02=25
span03=50

df['sma01']=price.rolling(window=span01).mean()
df['sma02']=price.rolling(window=span02).mean()
df['sma03']=price.rolling(window=span03).mean()

plt.figure(figsize=(30,15))
plt.subplot(2,1,1)
plt.plot(data,price,label='Close',color='#99b898')
plt.plot(data,df['sma01'],label='sma01',color='#e84a5f')
plt.plot(data,df['sma02'],label='sma02',color='#ff847c')
plt.plot(data,df['sma03'],label='sma03',color='#feceab')
plt.legend()

plt.subplot(2,1,2)
plt.bar(data,df['Volume'],label='Volume',color='grey')
plt.legend()

In [ ]:
pip install pandas_datareader

In [ ]:
from pandas_datareader import data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
def company_stock(start,end,company_code):
    df = data.DataReader(company_code,'stooq')
    df = df[(df.index>=start) & (df.index<=end)]



    data1=df.index
    price=df['Close']

    span01=5
    span02=25
    span03=50

    df['sma01']=price.rolling(window=span01).mean()
    df['sma02']=price.rolling(window=span02).mean()
    df['sma03']=price.rolling(window=span03).mean()

    plt.figure(figsize=(30,15))
    plt.subplot(2,1,1)
    plt.plot(data1,price,label='Close',color='#99b898')
    plt.plot(data1,df['sma01'],label='sma01',color='#e84a5f')
    plt.plot(data1,df['sma02'],label='sma02',color='#ff847c')
    plt.plot(data1,df['sma03'],label='sma03',color='#feceab')
    plt.legend()

    plt.subplot(2,1,2)
    plt.bar(data1,df['Volume'],label='Volume',color='grey')
    plt.legend()

In [ ]:
company_stock('2017-01-01','2020-06-01','7203.jp')